### Creates a spreadsheet with evaluations of the metadata based on whether it is in the required field. It is not up to date for the new data model which includes expansions to sample information and related datasets and software systems.

In [ ]:
# All required libraries
import openpyxl
from openpyxl import Workbook, styles
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import CellIsRule, ColorScaleRule, FormulaRule, IconSetRule
from openpyxl.utils import get_column_letter
import json
import os
import regex as re

In [ ]:
# Required fields as dictated by the data model
global required_fields
required_fields = ['dataset: title', 'dataset: description', 'related publication: publicationTitle', 
                   'related publication: publicationAuthor', 'related publication: publicationPublisher', 
                   'related publication: publicationDateOfPublication', 'sample: name', 'sample: description', 
                   'sample: porousMediaType', 'sample: source', 'digital dataset: name', 
                   'digital dataset: isSegmented', 'digital dataset: description','analysis dataset: isSegmented', 
                   'analysis dataset: name', 'analysis dataset: description', 'analysis dataset: datasetType']

global requirement_weight
requirement_weight = 1

In [ ]:
# Checks if a provided field is blank or not. If it has certain text that might be a researcher trying to avoid a required field, that is logged to the trips variable
def exists(field,name,index):
    global trips
    if field == None:
        return False
    elif field == "-" or field == "" or field == "N/A" or field == "n/a":
        trip = {"loc":filename,"field":name,"nodeNum":index,"input":field}
        trips.append(trip)
        return False
    else:
        return True

In [ ]:
# Checks all the fields for a dataset
def check_dataset():
    counter = {}
    fields = ["title","description","authors","publicationDate","doi","license"]
    for field in fields:
        if exists(metadata.get('nodes')[0].get("value").get(field),field,0):
            counter[field] = counter.get(field,0)+1
        else:
            counter[field] = counter.get(field,0)
            
    return counter

In [ ]:
# Checks all the fields for a related publication
def check_related_pubs():
    pubs = 0
    counter = {}
    fields = ["publicationTitle","publicationAuthor","publicationPublisher","publicationDateOfPublication","publicationLink",
             "publicationDescription"]
    for field in fields:
        counter[field]=0
    for pub in metadata.get('nodes')[0].get('value').get('relatedPublications'):
        pubs += 1
        for field in fields:
            if exists(pub.get(field),field,metadata.get('nodes')[0].get('value').get('relatedPublications').index(pub)):
                counter[field] = counter.get(field,0)+1
            else:
                counter[field] = counter.get(field,0)
    for key in counter.keys():
        try:
            counter[key] = counter[key]/pubs
        except ZeroDivisionError:
            counter[key] = "N/A"
    return counter

In [ ]:
# Checks all the fields for a sample
def check_sample():
    count = 0
    counter = {}
    fields = ["name","description","porousMediaType","source","geographicalLocation",'grainSizeAvg', 'grainSizeMax', 
              'grainSizeMin','porosity']
    for field in fields:
        counter[field]=0
    for node in metadata.get("nodes"):
        if node.get("value").get("dataType") == "sample":
            count+=1
            for field in fields:
                if exists(node.get("value").get(field),field,metadata.get('nodes').index(node)):
                    counter[field] = counter.get(field,0)+1
                else:
                    counter[field] = counter.get(field,0)
    for key in counter.keys():
        try:
            counter[key] = counter[key]/count
        except ZeroDivisionError:
            counter[key] = "N/A"
    return counter

In [ ]:
# Checks all the fields for a digital dataset
def check_digital_dataset():
    count = 0
    counter = {}
    fields = ["name","isSegmented","description","voxelX","voxelY","voxelZ","voxelUnits"]
    for field in fields:
        counter[field]=0
    for node in metadata.get("nodes"):
        if node.get("value").get("dataType") == "digital_dataset":
            count+=1
            for field in fields:
                if exists(node.get("value").get(field),field,metadata.get('nodes').index(node)):
                    counter[field] = counter.get(field,0)+1
                else:
                    counter[field] = counter.get(field,0)
    for key in counter.keys():
        try:
            counter[key] = counter[key]/count
        except ZeroDivisionError:
            counter[key] = "N/A"
    return counter

In [ ]:
# Checks all the fields for an analysis dataset
def check_analysis_dataset():
    count = 0
    counter = {}
    fields = ["isSegmented","name","description","datasetType"]
    for field in fields:
        counter[field]=0
    for node in metadata.get("nodes"):
        if node.get("value").get("dataType") == "analysis_data":
            count+=1
            for field in fields:
                if exists(node.get("value").get(field),field,metadata.get('nodes').index(node)):
                    counter[field] = counter.get(field,0)+1
                else:
                    counter[field] = counter.get(field,0)
    for key in counter.keys():
        try:
            counter[key] = counter[key]/count
        except ZeroDivisionError:
            counter[key] = "N/A"
    return counter

In [ ]:
# Looks through the fields of an entire dataset
def check_all():
    global averages
    dataset = check_dataset()
    sample = check_sample()
    related_pubs = check_related_pubs()
    digital_dataset = check_digital_dataset()
    analysis_dataset = check_analysis_dataset()
    data = [(dataset,"dataset:"),(related_pubs,"related publication:"),(sample,"sample:"),(digital_dataset,"digital dataset:"),
            (analysis_dataset,"analysis dataset:")]
    weights = []
    keys = []
    for element in data:
        weights.extend(list(element[0].values()))
        keys.extend([element[1] +' '+item for item in list(element[0].keys())])
    total = 0
    count = 0
    for key, weight in zip(keys, weights):
        if weight != 'N/A':
            if key in required_fields:
                total += requirement_weight*weight
                count += requirement_weight
            else:
                total += weight
                count += 1
    avg = total/count
    keys.append('totalScore')
    weights.append(avg)
    averages.append(avg)
    return keys, weights

In [ ]:
# Creates an Excel sheet with all the information on all datasets. It is also color coded with red as 0% of a field full and green as 100% of the field full

wb = Workbook()
ws = wb.active
# Weight given to required fields in the final average. 1 is equal weight, 0 is no weight, anything higher than 1 is that amount of weight greater.
requirement_weight = 2

global averages
averages = []

# Please put in the name of the filepath to all metadatafiles
filepath = "file/path/to/all/MetadataFiles"
directory = os.fsencode(filepath)
ls = check_all()
ws.append(["project number"]+ls[0])

global trips
trips = []

global filename
for file in os.listdir(directory):
    if os.fsdecode(file) != ".DS_Store":
        filename = os.fsdecode(file)
        full_filename = filepath+filename
        with open(full_filename, 'r') as file:
            metadata = json.load(file)
        ls = check_all()
        ws.append([int(re.findall(r'\d{1,3}',filename)[0])]+ls[1])
    else: 
        pass

color_rule = ColorScaleRule(start_type='num', start_value=0, start_color='FF9C9C',
                            mid_type='num', mid_value=.5, mid_color='FFFFFF',
                            end_type='num', end_value=1, end_color='6AA84F')

col = get_column_letter(len(ls[0]))
row = str(len(os.listdir(directory)))
scope = 'B2:'+col+row

ws.conditional_formatting.add(
    scope,
    color_rule
)

color_rule = ColorScaleRule(start_type='num', start_value=min(averages), start_color='FFFFFF',
                            # mid_type='num', mid_value=.5, mid_color='FFFFFF',
                            end_type='num', end_value=1, end_color='6AA84F')

col = get_column_letter(len(ls[0])+1)
scope = col+'2:'+col+row

ws.conditional_formatting.add(
    scope,
    color_rule
)

wb.save("MetadataEvaluation.xlsx")